# Real-time Workload for Scylla
### Web Sales | Web Returns | Store Sales | Store Returns | Catalog Sales

### Installing NoSQLBench 

#### Download:

In [1]:
!curl -L -O https://github.com/nosqlbench/nosqlbench/releases/latest/download/nb
!chmod +x nb

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   146  100   146    0     0    299      0 --:--:-- --:--:-- --:--:--   298
100   641  100   641    0     0    890      0 --:--:-- --:--:-- --:--:--   890
100  218M  100  218M    0     0  30.6M      0  0:00:07  0:00:07 --:--:-- 44.2M


### Getting variables values 

Importing stuff and starting Spark Context:

In [2]:
from cassandra.cluster import Cluster, ExecutionProfile, EXEC_PROFILE_DEFAULT
from cassandra.policies import DCAwareRoundRobinPolicy, TokenAwarePolicy, DowngradingConsistencyRetryPolicy, ConsistencyLevel, RoundRobinPolicy
from cassandra.query import tuple_factory
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkContext
from pyspark.sql import functions as F
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, FloatType, DateType, LongType

## Starting Spark
spark = SparkSession\
    .builder\
    .appName("TPCDS-Scylla")\
    .config("setMaster","172.19.0.2")\
    .config("spark.jars", "target/scala-2.12/spark3-scylla4-example-assembly-0.1.jar")\
    .config("spark.cassandra.connection.host", "172.19.0.2")\
    .config('spark.cassandra.output.consistency.level','LOCAL_QUORUM')\
    .config("spark.driver.memory", "28g")\
    .config("spark.executor.memory", "28g")\
    .getOrCreate()
sc = spark.sparkContext
## Start SQL Context, it will enable you to run SQL Queries
sqlContext = SQLContext(spark)

21/11/30 12:34:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Creating Spark Tables based on ScyllaDB

In [4]:
call_center = spark.read.format("org.apache.spark.sql.cassandra").options(table="call_center", keyspace="tpcds").load()
call_center.registerTempTable("call_center")
catalog_page = spark.read.format("org.apache.spark.sql.cassandra").options(table="catalog_page", keyspace="tpcds").load()
catalog_page.registerTempTable("catalog_page")
catalog_returns = spark.read.format("org.apache.spark.sql.cassandra").options(table="catalog_returns", keyspace="tpcds").load()
catalog_returns.registerTempTable("catalog_returns")
catalog_sales = spark.read.format("org.apache.spark.sql.cassandra").options(table="catalog_sales", keyspace="tpcds").load()
catalog_sales.registerTempTable("catalog_sales")
customer = spark.read.format("org.apache.spark.sql.cassandra").options(table="customer", keyspace="tpcds").load()
customer.registerTempTable("customer")
customer_address = spark.read.format("org.apache.spark.sql.cassandra").options(table="customer_address", keyspace="tpcds").load()
customer_address.registerTempTable("customer_address")
customer_demographics = spark.read.format("org.apache.spark.sql.cassandra").options(table="customer_demographics", keyspace="tpcds").load()
customer_demographics.registerTempTable("customer_demographics")
date_dim = spark.read.format("org.apache.spark.sql.cassandra").options(table="date_dim", keyspace="tpcds").load()
date_dim.registerTempTable("date_dim")
household_demographics = spark.read.format("org.apache.spark.sql.cassandra").options(table="household_demographics", keyspace="tpcds").load()
household_demographics.registerTempTable("household_demographics")
income_band = spark.read.format("org.apache.spark.sql.cassandra").options(table="income_band", keyspace="tpcds").load()
income_band.registerTempTable("income_band")
inventory = spark.read.format("org.apache.spark.sql.cassandra").options(table="inventory", keyspace="tpcds").load()
inventory.registerTempTable("inventory")
item = spark.read.format("org.apache.spark.sql.cassandra").options(table="item", keyspace="tpcds").load()
item.registerTempTable("item")
promotion = spark.read.format("org.apache.spark.sql.cassandra").options(table="promotion", keyspace="tpcds").load()
promotion.registerTempTable("promotion")
reason = spark.read.format("org.apache.spark.sql.cassandra").options(table="reason", keyspace="tpcds").load()
reason.registerTempTable("reason")
ship_mode = spark.read.format("org.apache.spark.sql.cassandra").options(table="ship_mode", keyspace="tpcds").load()
ship_mode.registerTempTable("ship_mode")
store = spark.read.format("org.apache.spark.sql.cassandra").options(table="store", keyspace="tpcds").load()
store.registerTempTable("store")
store_returns = spark.read.format("org.apache.spark.sql.cassandra").options(table="store_returns", keyspace="tpcds").load()
store_returns.registerTempTable("store_returns")
store_sales = spark.read.format("org.apache.spark.sql.cassandra").options(table="store_sales", keyspace="tpcds").load()
store_sales.registerTempTable("store_sales")
time_dim = spark.read.format("org.apache.spark.sql.cassandra").options(table="time_dim", keyspace="tpcds").load()
time_dim.registerTempTable("time_dim")
warehouse = spark.read.format("org.apache.spark.sql.cassandra").options(table="warehouse", keyspace="tpcds").load()
warehouse.registerTempTable("warehouse")
web_page = spark.read.format("org.apache.spark.sql.cassandra").options(table="web_page", keyspace="tpcds").load()
web_page.registerTempTable("web_page")
web_returns = spark.read.format("org.apache.spark.sql.cassandra").options(table="web_returns", keyspace="tpcds").load()
web_returns.registerTempTable("web_returns")
web_sales = spark.read.format("org.apache.spark.sql.cassandra").options(table="web_sales", keyspace="tpcds").load()
web_sales.registerTempTable("web_sales")
web_site = spark.read.format("org.apache.spark.sql.cassandra").options(table="web_site", keyspace="tpcds").load()
web_site.registerTempTable("web_site")



21/11/30 12:39:03 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


## Creating Workload variables based on dimension tables size already loaded into scyllaDB

Install requirements:

In [7]:
!pip install pyyaml

Reading template file:

In [8]:
import yaml

with open(r'workload-nosqlbench.template.yaml') as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    workload = yaml.load(file, Loader=yaml.FullLoader)
    columns = workload["bindings"]
 


Since the size of the environment can change, we need to check what are the Max keys that can be used to generate data that it will return data from queries. For this, we will be using Spark to get the MAX of each key.

In [9]:
column_to_get_max=[]
variables_values=[]
import re
for column,val in columns.items():
    if column.endswith("_sk"):
        column_to_get_max.append([column,val])
        print([column,val])
    else:
        pass
 

['wr_item_sk', 'Uniform(0,<<max_item_sk>>)']
['wr_web_page_sk', '<<max_web_page_sk>>']
['wr_returned_date_sk', 'Uniform(<<start_date>>,<<end_date>>)']
['wr_returned_time_sk', 'Uniform(0,86399)']
['wr_returning_addr_sk', 'Uniform(0,<<max_c_address_sk>>)']
['wr_returning_cdemo_sk', 'Uniform(0,<<max_cdemo_sk>>)']
['wr_returning_customer_sk', 'Uniform(<<max_customer_sk>>))']
['wr_refunded_cdemo_sk', 'Uniform(0,<<max_cdemo_sk>>)']
['wr_refunded_customer_sk', 'Uniform(<<max_customer_sk>>))']
['wr_refunded_hdemo_sk', 'Uniform(0,7200)']
['wr_returning_hdemo_sk', 'Uniform(0,7200)']
['wr_refunded_addr_sk', 'Uniform(0,<<max_c_address_sk>>)']
['wr_reason_sk', 'Uniform(0,<<max_reason_sk>>)']
['ws_item_sk', 'Uniform(0,<<max_item_sk>>)']
['ws_sold_date_sk', 'Uniform(<<start_date>>,<<end_date>>)']
['ws_sold_time_sk', 'Uniform(0,86399)']
['ws_warehouse_sk', 'Uniform(0,<<max_warehouse>>)']
['ws_ship_date_sk', 'Uniform(<<start_date>>,<<end_date>>)']
['ws_bill_customer_sk', 'Uniform(<<max_customer_sk>>))'

Capturing only the keys names from the template file:

In [10]:
# column_to_get_max 
full_variables_list = []
for column,value in column_to_get_max:
    if re.findall('<<(.+?)>>', value):
        match = re.findall('<<(.+?)>>', value)
        full_variables_list.append(match)
        #print(match)
    else: 
        pass
variable_list=[]
[variable_list.append(x) for x in full_variables_list if x not in variable_list]
print(variable_list)

[['max_item_sk'], ['max_web_page_sk'], ['start_date', 'end_date'], ['max_c_address_sk'], ['max_cdemo_sk'], ['max_customer_sk'], ['max_reason_sk'], ['max_warehouse'], ['max_promo_sk'], ['max_web_site_sk'], ['max_store_sk'], ['max_warehouse_sk'], ['max_call_center_sk'], ['max_catalog_page_sk']]


Creating relationship between foreign keys, tables and variables


In [11]:
array = [["cp_catalog_page_sk","catalog_page",'max_catalog_page_sk'],["cc_call_center_sk","call_center","max_call_center_sk"],["s_store_sk","store","max_store_sk"],["p_promo_sk","promotion",'max_promo_sk'],["r_reason_sk","reason",'max_reason_sk'],["i_item_sk", "item","max_item_sk"],["c_customer_sk","customer","max_customer_sk"],["cd_demo_sk","customer_demographics","max_cdemo_sk"],["ca_address_sk","customer_address","max_c_address_sk"],["wp_web_page_sk","web_page","max_web_page_sk"],["web_site_sk","web_site","max_web_site_sk"],["w_warehouse_sk","warehouse","max_warehouse"]]


 Generating and executing queries and capturing the values

In [12]:

#['start_date', 'end_date'], [''],  , [''], [''], [''], ['max_call_center_sk'], 
queries=[]
for column, tables,variable in array:
    query = 'select max({}) as max from {}'.format(column,tables)
    
    result = sqlContext.sql(query).collect()[0]
    print(query + " = " + str(result[0]) + "-> {}".format(variable))
    print("result: "+ str(result[0]) + " | column: " + column + "| variable :" + variable)
    queries.append([result[0],column,variable])



select max(cp_catalog_page_sk) as max from catalog_page = 11718-> max_catalog_page_sk
result: 11718 | column: cp_catalog_page_sk| variable :max_catalog_page_sk
select max(cc_call_center_sk) as max from call_center = 6-> max_call_center_sk
result: 6 | column: cc_call_center_sk| variable :max_call_center_sk
select max(s_store_sk) as max from store = 12-> max_store_sk
result: 12 | column: s_store_sk| variable :max_store_sk
select max(p_promo_sk) as max from promotion = 300-> max_promo_sk
result: 300 | column: p_promo_sk| variable :max_promo_sk
select max(r_reason_sk) as max from reason = 35-> max_reason_sk
result: 35 | column: r_reason_sk| variable :max_reason_sk
select max(i_item_sk) as max from item = 18000-> max_item_sk
result: 18000 | column: i_item_sk| variable :max_item_sk


select max(c_customer_sk) as max from customer = 1920800-> max_customer_sk
result: 1920800 | column: c_customer_sk| variable :max_customer_sk


select max(cd_demo_sk) as max from customer_demographics = 1920800-> max_cdemo_sk
result: 1920800 | column: cd_demo_sk| variable :max_cdemo_sk
select max(ca_address_sk) as max from customer_address = 50000-> max_c_address_sk
result: 50000 | column: ca_address_sk| variable :max_c_address_sk
select max(wp_web_page_sk) as max from web_page = 60-> max_web_page_sk
result: 60 | column: wp_web_page_sk| variable :max_web_page_sk
select max(web_site_sk) as max from web_site = 30-> max_web_site_sk
result: 30 | column: web_site_sk| variable :max_web_site_sk
select max(w_warehouse_sk) as max from warehouse = 5-> max_warehouse
result: 5 | column: w_warehouse_sk| variable :max_warehouse


#### Now if that we have values of each variable, we will need to create the file replacing the variables for the values

In [15]:
## ASK HELP FOR THE TEAM TO DUMP THE NEW VALUES INTO A NEW FILE REPLACING THE STRINGS
! pip3 install pandasql

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26781 sha256=3fed724425e945e399fed80b223e62f34e4975e715a81ec0455cd27c400cab9c
  Stored in directory: /home/jovyan/.cache/pip/wheels/63/e8/ec/75b1df467ecf57b6ececb32cb16f4e86697cbfe55cb0c51f07
Successfully built pandasql


In [27]:
# queries.append([result[0],column,variable])
i=0
import yaml
import numpy as np 
import pandas as pd
new_file=[]
with open(r'workload-nosqlbench.template.yaml') as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    workload = yaml.load(file, Loader=yaml.FullLoader)
import pandasql as psql
  
columns = workload["bindings"]

df_values = pd.DataFrame(queries,columns=['result',"column","variable"])
df = pd.DataFrame(columns.items(),columns=["column","value"])
df['parsed'] = df.loc[df['value'].apply(lambda st: st[st.find("<")+2:st.find(">")]).isin(df_values.variable)== True,df['parsed']] = df['value'].apply(lambda st: st[st.find("<")+2:st.find(">")])
df = df[["column","value","parsed"]]
df['column'] = df['column'].astype('string')
df_values['column'] = df_values['column'].astype('string')

new_df = psql.sqldf("select distinct df.column,df.value,cast(df_values.result as text) as result from df_values  join df on cast(df_values.column as string)= cast(df.column as string) and parsed=variable") 

pat = re.compile(r"<<.*?>>")

new_df["value"] = new_df[["value","result"]].apply(lambda x: pat.sub(repl=x[1],string=x[0]),axis=1)

new_df


,column,value,result
0,cs_catalog_page_sk,"Uniform(0,11718)",11718
1,cs_call_center_sk,"Uniform(0,6)",6
2,sr_store_sk,"Uniform(0,12))",12
3,ss_store_sk,"Uniform(0,12))",12
4,cs_promo_sk,"Uniform(0,300)",300
5,ss_promo_sk,"Uniform(0,300)",300
6,ws_promo_sk,"Uniform(0,300)",300
7,sr_reason_sk,"Uniform(0,35)",35
8,wr_reason_sk,"Uniform(0,35)",35
9,cs_item_sk,"Uniform(0,18000)",18000


In [67]:
import ruamel.yaml
import sys
yaml = ruamel.yaml.YAML()
import re


# #new_df.info()
# #print(type(new_df))
file = 'workload-nosqlbench.template.yaml'
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    #workload = yaml.load(file)
config, ind, bsi = ruamel.yaml.util.load_yaml_guess_indent(open(file))

#columns_pop = config['bindings']
# instances[0]['host'] = '1.2.3.4'
# instances[0]['username'] = 'Username'
# instances[0]['password'] = 'Password'

yaml = ruamel.yaml.YAML()
yaml.indent(mapping=ind, sequence=ind, offset=bsi) 
with open('output.yaml', 'w') as fp:
    yaml.dump(config, fp)
    
    
    
# data['bindings'].items()
#data['bindings'].values()
pat = re.compile("<<(.*?)>>")
#new_df.to_dict
#for dct in lst:
#df = workload['bindings'].items()

for key,value in config['bindings'].items():
    #print(key,value)
    #print(workload['bindings'][item])
    for column,new_value,result in new_df.values:
        #print(column,new_value)
        if key==column:
            print("match")
            #print(type(columns_pop[key]))
            #value =  new_value
            config['bindings'][key] = new_value
            print(config['bindings'][key])
            #print(key,new_value)
            #level1[1] == level1[1].replace(column,level1[1])
            #print(workload['bindings']+": ",key)

  #  print(level1[0],": ",level1[1])
        
        


match
Uniform(0,18000)
match
60
match
Uniform(0,50000)
match
Uniform(0,1920800)
match
Uniform(1920800))
match
Uniform(0,1920800)
match
Uniform(1920800))
match
Uniform(0,50000)
match
Uniform(0,35)
match
Uniform(0,18000)
match
Uniform(0,5)
match
Uniform(1920800))
match
Uniform(0,1920800)
match
Uniform(0,50000)
match
Uniform(1920800))
match
Uniform(0,1920800)
match
Uniform(0,50000)
match
60
match
Uniform(0,300)
match
Uniform(0,30)
match
Uniform(0,18000)
match
Uniform(0,12))
match
Uniform(0,1920800)
match
Uniform(1920800))
match
Uniform(0,50000)
match
Uniform(0,35)
match
Uniform(0,18000)
match
Uniform(0,50000)
match
Uniform(0,1920800)
match
Uniform(0,12))
match
Uniform(1920800))
match
Uniform(0,300)
match
Uniform(0,18000)
match
Uniform(0,50000)
match
Uniform(0,1920800)
match
Uniform(0,1920800)
match
Uniform(0,50000)
match
Uniform(0,1920800)
match
Uniform(0,1920800)
match
Uniform(0,6)
match
Uniform(0,11718)
match
Uniform(0,300)


In [69]:
with open('names.yaml', 'w') as file:
    yaml.dump(config, file)

In [193]:
import jinja2
from jinja2 import meta
templateLoader = jinja2.FileSystemLoader(searchpath="./")
templateEnv = jinja2.Environment(loader=templateLoader)
TEMPLATE_FILE = "workload-nosqlbench.template.yaml"
template = templateEnv.get_template(TEMPLATE_FILE)
#outputText = template.render()
parsed_content = templateEnv.parse(template)
jinja2.meta.find_undeclared_variables(parsed_content)

set()

## Using  NoSQLBench against ScyllaDB

cycles - standard, however the cql activity type will default this to however many statements are included in the current activity, after tag filtering, etc.

pooling default: none - Applies the connection pooling options to the policy. Examples:

    pooling=4:10 keep between 4 and 10 connections to LOCAL hosts
    pooling=4:10,2:5 keep 4-10 connections to LOCAL hosts and 2-5 to REMOTE
    pooling=4:10:2000 keep between 4-10 connections to LOCAL hosts with up to 2000 requests per connection
    pooling=5:10:2000,2:4:1000 keep between 5-10 connections to LOCAL hosts with up to 2000 requests per connection, and 2-4 connection to REMOTE hosts with up to 1000 requests per connection
    
    
Additionally, you may provide the following options on pooling. Any of these that are provided must appear in this order: ,heartbeat_interval_s:n,idle_timeout_s:n,pool_timeout_ms:n, so a full example with all options set would appear as:

       pooling=5:10:2000,2:4:1000,heartbeat_interval_s:30,idle_timeout_s:120,pool_timeout_ms:5

lbp - configures the load balancing policies for the Java driver. With this parameter, you can configure nested load balancing policies in short-hand form.

The policies available are documented in detail under the help topic cql-loadbalancing. See that guide if you need more than the examples below.

Examples:
    

        lbp=LAP(retry_period=3,scale=10) - Latency aware policy with retry period of 3 seconds. (Seconds is the default time unit, unless _ms parameter is used) and scale 10.
        lbp=LAP(rp=3,s=10) - Same as above, using the equivalent but terser form.
        lbp=LAP(rp_ms=3000,s_ms=10000) - Same as above, with milliseconds instead of seconds.
        loadbalancing=LAP(s=10),TAP() - Latency aware policy, followed by token aware policy.

#### Start Schema 

In [5]:
!./nb run driver=cqld3 \
    host=172.19.0.2\
    workload=workload-nosqlbench.yaml\
    tags=phase:schema \
    cycles=11 \
    loadbalancing='TAP()'


### Start workload

In [6]:
!./nb run driver=cqld3 \
        host=172.19.0.2\
        workload=workload-nosqlbench.yaml\
        threads=auto \
        cycles=3000000 \
        async=64  \
        tags=phase:rampup \
        --progress console:10s \
        loadbalancing='TAP()' pooling=8:16:10000,8:16:10000  
                        #start_connections:number_max_connections:ops per connection ## 2*shards:4*shards:5000

workload-nosqlbench.yaml: 6.66%/Running (details: min=0 cycle=199680 max=3000000)
workload-nosqlbench.yaml: 17.91%/Running (details: min=0 cycle=537440 max=3000000)
workload-nosqlbench.yaml: 29.14%/Running (details: min=0 cycle=874112 max=3000000)
workload-nosqlbench.yaml: 39.60%/Running (details: min=0 cycle=1187968 max=3000000)
workload-nosqlbench.yaml: 48.47%/Running (details: min=0 cycle=1454080 max=3000000)
   60545 ERROR [cluster1-nio-worker-1] NBCycleErrorHandler error with cycle 1614867: statement: (prepared) insert into tpcds.store_sales 
(ss_item_sk,ss_ticket_number,ss_addr_sk,ss_cdemo_sk,ss_coupon_amt,ss_customer_sk,ss_ext_discount_amt,ss_ext_list_price,ss_ext_sales_price,ss_ext_tax,ss_ext_wholesale_cost,ss_hdemo_sk,ss_list_price,ss_net_paid,ss_net_paid_inc_tax,ss_net_profit,ss_promo_sk,ss_quantity,ss_sales_price,ss_sold_date_sk,ss_sold_time_sk,ss_store_sk,ss_wholesale_cost)
values
(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
 VALUES[236923,840874,131624,1615152,9979.6397